In [4]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from numpy.linalg import pinv

In [5]:
class LogicalRegression:
    
    def __init__(self, X=None, Y=None):
        
        if X is None:
            raise TypeError("'X' expected as 2-dimensional array, found 'None' instead")
        else:
            temp = np.array(X)
            if temp.ndim == 1:
                raise TypeError("'X' is 1-dimensional array, not compatible with 2-dimension matrices")
            if temp.ndim > 2:
                raise TypeError("'X' cannot be more than 2-dimensional, please specify a 2d matrix")
            
            self.X = np.concatenate((np.ones((len(temp), 1)), temp), axis=1)
            
        if Y is None:
            raise TypeError("'Y' expected as 1-dimensional array, found 'None' instead")
        else:
            temp = np.array(Y)
            if temp.ndim > 2:
                raise TypeError("'Y' cannot be more than 2-dimensional, please specify a 1d vector")
            if temp.ndim == 1:
                temp = temp.reshape((-1, 1))
            elif temp.shape[0] == 1:
                temp = temp.T
            elif not temp.shape[1] == 1:
                raise TypeError("'Y' has to be a 1d-array or a 2-dimensional vector along the row or column")
            
            self.Y = temp
            
            if not len(self.X) == len(self.Y):
                raise RuntimeError("Length of input records in 'X' %d do not match to number of length of output \
                        vector %d" % (self.X.shape[0],self.Y.shape[0]))
        
        self.features = self.X.shape[1]
        self.num = self.X.shape[0]
        
#         self.theta = np.zeros((self.features, 1))
        self.theta = np.random.rand(self.features, 1)
    
#         self.validate()
    
    def validate(self):
        pass
    
    def hypothesis(self, theta=None, X=None, use_model=True):
        
        if use_data:
            theta = self.theta
            X = self.X
        
        if theta is None:
            raise TypeError("'theta' is expected as 1-dimensional array, found 'None' instead")
            
        if X is None:
            raise TypeError("'X' is expected as 2-dimensional array or matrix, found 'None' instead")
        
        temp = np.dot(X, theta)
        return 1 / (1 + np.exp(-1 * temp))
    
    def G(self, **kwargs):
        return hypothesis(**kwargs)
    
    def cost_function(self, X=None, Y=None, use_model=True, regularize=False, reg_lambda=0):
        
        if use_model:
            X = self.X
            Y = self.Y
                
        if X is None:
            raise TypeError("'X' expected as 2-dimensional array or matrix, found 'None' instead")
        elif not X.ndim == 2:
            raise TypeError("'X' expected as 2-dimensional array or matrix, not " + X.ndim + " dimensions")
            
        if Y is None:
            raise TypeError("'Y' expected as 1-dimensional vector, found 'None' instead")
        elif Y.ndim == 2:
            if 1 not in Y.shape:
                raise TypeError("'Y' expected as 2-dimensional row or column vector, not " + str(Y.shape) + 
                                " dimensions")
            else:
                Y = Y.reshape(-1, 1)
        elif Y.ndim == 1:
            Y = Y.reshape(-1, 1)
        
        H = hypothesis(theta=self.theta, X=X, use_model=use_model)
        m = X.shape[0]
        
        temp = Y.dot(np.log(H)) + (1-Y).dot(np.log(1-H))
        j = -1 * temp.sum() / m
        
        if regularize:
            reg = reg_lambda * (theta**2).sum()
            return j + reg / 2 / m
        else:
            return j
            
    def J(self, **kwargs):
        return cost_function(**kwargs)
    
    def gradient_descent(self, X=None, Y=None, alpha=0, use_model=False, regularize=False, reg_lambda=0):
        
        if use_model:
            X = self.X
            Y = self.Y
                
        if X is None:
            raise TypeError("'X' expected as 2-dimensional array or matrix, found 'None' instead")
        elif not X.ndim == 2:
            raise TypeError("'X' expected as 2-dimensional array or matrix, not " + X.ndim + " dimensions")
            
        if Y is None:
            raise TypeError("'Y' expected as 1-dimensional vector, found 'None' instead")
        elif Y.ndim == 2:
            if 1 not in Y.shape:
                raise TypeError("'Y' expected as 2-dimensional row or column vector, not " + str(Y.shape) + 
                                " dimensions")
            else:
                Y = Y.reshape(-1, 1)
        elif Y.ndim == 1:
            Y = Y.reshape(-1, 1)
            
        m = X.shape[0]
        
        temp = np.dot(X, self.theta)
        temp = temp - Y
        
        if regularize:
            return ((1 - alpha*reg_lambda/m) * self.theta) - (alpha * (X.T.dot(temp)) / m)
        else:
            return self.theta - (alpha * (X.T.dot(temp)) / m)
    
    def normal_theta(self, ip=None, op=None, use_model=False, regularize=False, reg_lambda=0):
        
        if use_model:
            ip = self.X
            op = self.Y
        
        if ip is None:
            raise TypeError("'ip' expected as 2-dimensional array or matrix, found 'None' instead")
        
        if not isinstance(ip, np.ndarray):
            ip = np.array(ip)
            
        if not ip.ndim == 2:
            raise TypeError("'ip' expected as 2-dimensional array or matrix, not" + str(ip.ndim) + " dimensions")
            
        if op is None:
            raise TypeError("'op' expected as 1-dimensional vector, found 'None' instead")
        
        if not isinstance(op, np.ndarray):
            op = np.array(op)
        
        if op.ndim == 2:
            if 1 not in op.shape:
                raise TypeError("'op' expected as 2-dimensional row or column vector, not " + str(op.shape) + 
                                " dimensions")
            else:
                op = op.reshape(-1, 1)
        elif Y.ndim == 1:
            op = op.reshape(-1, 1)
        
        A = pinv(ip.T.dot(ip))
        n = ip.shape[1]
        i = np.eye(n)
        i[0][0] = 0
        
        B = A
        if regularize:
            B = B + reg_lambda*i
        return B.dot(ip.T).dot(op)
    
    def fit(self, use_GD=False, regularize=False, reg_lambda=0, loop=1000):
        if not use_GD:
            self.theta = self.normal_theta(use_model=True, regularize=regularize, reg_lambda=reg_lambda)
        else:
            for i in range(loop):
                self.theta = self.gradient_descent(use_model=True, regularize=regularize, \
                                                   reg_lambda=reg_lambda, alpha=.03)
        
        return self.theta
    
    def predict(self, ip=None, threshold=.5):
        
        ip = np.array(ip)
        
        if ip.ndim == 1:
            ip = ip.reshape(1, -1)
        if ip.ndim <= 2:
            ip = np.concatenate((np.ones((len(ip), 1)), ip), axis=1)
        if ip.ndim > 2:
            raise TypeError("'ip' has to be a 1d-vector or a 2-dimensional array or matrix")
            
        return ip.dot(self.theta) >= threshold
    
    def get_nfeature(self):
        return self.X.shape[1]-1
        

In [6]:
model = LogicalRegression(X=[[1], [2], [3], [4], [5]], Y=[0, 0, 0, 1, 1])

In [7]:
param = model.fit(use_GD=True)
param

array([[-0.49607818],
       [ 0.29891372]])

In [8]:
model.theta = param
model.predict(ip=[4])

array([[ True]], dtype=bool)

In [9]:
model.get_nfeature()

1